In [3]:
from utils import *
import numpy as np
import multiprocessing as mp
import os

UNITY_APP_PATH = "/home/max/Documents/Unity/guidewire-simulation-transversal-verlet.x86_64"
SAVE_PATH = "/home/max/Temp/Praktikum/transversal"

REPEAT = 3 # Repeat each experiment REPEAT times
MP_CPUS = 8 # Number of CPUs to use for multiprocessing
NICENESS = 1 # Niceness of the Unity process


# Test experiment script

In [4]:
parameter_path = "/home/max/Github/guidewire-simulation/Scripts/sample-parameters.json"
log_path = "/home/max/Github/guidewire-simulation/Scripts/example-log.txt"


run_unity(UNITY_APP_PATH, parameter_path, log_path, headless=False)

Running Unity with the following command:
/home/max/Documents/Unity/guidewire-simulation-transversal-verlet.x86_64 -parameters /home/max/Github/guidewire-simulation/Scripts/sample-parameters.json -logFile /home/max/Github/guidewire-simulation/Scripts/example-log.txt -monitor 2
Unity process finished with exit code: -11


CompletedProcess(args=['/home/max/Documents/Unity/guidewire-simulation-transversal-verlet.x86_64', '-parameters', '/home/max/Github/guidewire-simulation/Scripts/sample-parameters.json', '-logFile', '/home/max/Github/guidewire-simulation/Scripts/example-log.txt', '-monitor', '2'], returncode=-11)

# Read initial JSON parameters file

In [5]:
parameters = read_json_file("/home/max/Github/guidewire-simulation/Scripts/sample-parameters.json")

for key in parameters.keys():
    print(f"{key}: {parameters[key]}")

logFilePath: /home/max/Temp/Praktikum/experiments/
constraintSolverSteps: 500
timeStep: 0.001
displacement: 1.0
rodElementLength: 10.0
guidewireLength: 300.0
numberRodElements: 30
totalMass: 1.0
guidewireOffset: -100.0
position: {'x': 0.0, 'y': 0.0, 'z': 0.0}
scale: {'x': 1.0, 'y': 1.0, 'z': 1.0}
rotation: {'x': 0.0, 'y': 0.0, 'z': 0.0}
bendStiffness: 0.1
stretchStiffness: 0.1
sphereRadius: 5.0
collisionMargin: 5
collisionStiffness: 1e-05
deltaThreshold: 0.0
damping: 0.0
VerletIntegration: True
SteadyState: True


# Reference solution

In [8]:
parameters["numberRodElements"] = 100
parameters["timeStep"] = 0.00001
parameters["displacement"] = 2

experiment_folder, name = prepare_experiment(SAVE_PATH, f"reference")

run_expriment(UNITY_APP_PATH, experiment_folder, name, parameters)

Running Unity with the following command:
/home/max/Documents/Unity/guidewire-simulation-transversal-verlet.x86_64 -parameters /home/max/Temp/Praktikum/transversal/parameters.json -logFile /home/max/Temp/Praktikum/transversal/log.txt -monitor 2
Unity process finished with exit code: -11


# Constraint solver steps

In [9]:
parameters = read_json_file("/home/max/Github/guidewire-simulation/Scripts/sample-parameters.json")

constraint_solver_steps = np.arange(100,1100,100)#[100, 200, 300, 400, 500]

# repeat ten times to get some statistics
for i in range(REPEAT):
    experiment_folder, name = prepare_experiment(SAVE_PATH, f"constraint_solver_steps_{i}")

    def temp_fun(css):
        parametes_new = parameters.copy()
        parametes_new["constraintSolverSteps"] = int(css)
        run_expriment(UNITY_APP_PATH, experiment_folder, name+f"_{css}", parametes_new)

    with mp.Pool(MP_CPUS, initializer=set_nice, initargs=(NICENESS,)) as p:
        p.map(temp_fun, constraint_solver_steps)

Set niceness to:  1Set niceness to:  
Set niceness to: 1 
  
1Set niceness to: 
  1
 Set niceness to: 


 1Set niceness to:   
1
Set niceness to:   1
 


Set niceness to:  Running Unity with the following command:Running Unity with the following command:Running Unity with the following command:Running Unity with the following command:Running Unity with the following command:Running Unity with the following command:Running Unity with the following command:1




 
/home/max/Documents/Unity/guidewire-simulation-transversal-verlet.x86_64 -parameters /home/max/Temp/Praktikum/transversal_400/parameters.json -logFile /home/max/Temp/Praktikum/transversal_400/log.txt -monitor 2
/home/max/Documents/Unity/guidewire-simulation-transversal-verlet.x86_64 -parameters /home/max/Temp/Praktikum/transversal_200/parameters.json -logFile /home/max/Temp/Praktikum/transversal_200/log.txt -monitor 2/home/max/Documents/Unity/guidewire-simulation-transversal-verlet.x86_64 -parameters /home/max/Temp/Praktikum/tr

KeyboardInterrupt: 

# Time steps

In [12]:
parameters = read_json_file("/home/max/Github/guidewire-simulation/Scripts/sample-parameters.json")

timesteps = np.logspace(-4, 0, 10)


for i in range(REPEAT):
    experiment_folder, name = prepare_experiment(f"time_steps_{i}")
    
    def temp_fun(t):
        parametes_new = parameters.copy()
        parametes_new["timeStep"] = t
        run_expriment(UNITY_APP_PATH, experiment_folder, name + f"_{t}", parametes_new)

    with mp.Pool(MP_CPUS, initializer=set_nice, initargs=(NICENESS,)) as p:
        p.map(temp_fun, timesteps)

Running Unity with the following command:
/home/max/Documents/Unity/guidewire-simulation-longitudinal-verlet.x86_64 -parameters /home/max/Temp/Praktikum/longitudinal-verlet/time_steps_0/time_steps_0_0.0001/parameters.json -logFile /home/max/Temp/Praktikum/longitudinal-verlet/time_steps_0/time_steps_0_0.0001/log.txt -monitor 2
Unity process finished with exit code: -11
Running Unity with the following command:
/home/max/Documents/Unity/guidewire-simulation-longitudinal-verlet.x86_64 -parameters /home/max/Temp/Praktikum/longitudinal-verlet/time_steps_0/time_steps_0_0.0002782559402207126/parameters.json -logFile /home/max/Temp/Praktikum/longitudinal-verlet/time_steps_0/time_steps_0_0.0002782559402207126/log.txt -monitor 2
Unity process finished with exit code: -11
Running Unity with the following command:
/home/max/Documents/Unity/guidewire-simulation-longitudinal-verlet.x86_64 -parameters /home/max/Temp/Praktikum/longitudinal-verlet/time_steps_0/time_steps_0_0.000774263682681127/paramete

# Numer of Rod Elements

In [14]:
parameters = read_json_file("/home/max/Github/guidewire-simulation/Scripts/sample-parameters.json")


elements = np.arange(10,110,10).astype("float")

for i in range(REPEAT):
    experiment_folder, name = prepare_experiment(f"rod_elements_{i}")

    def temp_fun(e):
        parametes_new = parameters.copy()
        parametes_new["numberRodElements"] = e
        run_expriment(UNITY_APP_PATH, experiment_folder, name + f"_{e}", parametes_new)

    with mp.Pool(MP_CPUS, initializer=set_nice, initargs=(NICENESS,)) as p:
        p.map(temp_fun, elements)

Running Unity with the following command:
/home/max/Documents/Unity/guidewire-simulation-longitudinal-verlet.x86_64 -parameters /home/max/Temp/Praktikum/longitudinal-verlet/rod_elements_0/rod_elements_0_5.0/parameters.json -logFile /home/max/Temp/Praktikum/longitudinal-verlet/rod_elements_0/rod_elements_0_5.0/log.txt -monitor 2
Unity process finished with exit code: -11
Running Unity with the following command:
/home/max/Documents/Unity/guidewire-simulation-longitudinal-verlet.x86_64 -parameters /home/max/Temp/Praktikum/longitudinal-verlet/rod_elements_0/rod_elements_0_6.0/parameters.json -logFile /home/max/Temp/Praktikum/longitudinal-verlet/rod_elements_0/rod_elements_0_6.0/log.txt -monitor 2
Unity process finished with exit code: -11
Running Unity with the following command:
/home/max/Documents/Unity/guidewire-simulation-longitudinal-verlet.x86_64 -parameters /home/max/Temp/Praktikum/longitudinal-verlet/rod_elements_0/rod_elements_0_7.0/parameters.json -logFile /home/max/Temp/Praktik

# Displacement

In [16]:
parameters = read_json_file("/home/max/Github/guidewire-simulation/Scripts/sample-parameters.json")

elements = np.linspace(1,10, 10)


for i in range(REPEAT):
    experiment_folder, name = prepare_experiment(f"displacemen_{i}")

    def temp_fun(e):
        parametes_new = parameters.copy()
        parametes_new["displacement"] = e
        run_expriment(UNITY_APP_PATH, experiment_folder, name + f"_{e}", parametes_new)

    with mp.Pool(MP_CPUS, initializer=set_nice, initargs=(NICENESS,)) as p:
        p.map(temp_fun, elements)

Running Unity with the following command:
/home/max/Documents/Unity/guidewire-simulation-longitudinal-verlet.x86_64 -parameters /home/max/Temp/Praktikum/longitudinal-verlet/displacemen_0/displacemen_0_0.1/parameters.json -logFile /home/max/Temp/Praktikum/longitudinal-verlet/displacemen_0/displacemen_0_0.1/log.txt -monitor 2
Unity process finished with exit code: -11
Running Unity with the following command:
/home/max/Documents/Unity/guidewire-simulation-longitudinal-verlet.x86_64 -parameters /home/max/Temp/Praktikum/longitudinal-verlet/displacemen_0/displacemen_0_0.35789473684210527/parameters.json -logFile /home/max/Temp/Praktikum/longitudinal-verlet/displacemen_0/displacemen_0_0.35789473684210527/log.txt -monitor 2
Unity process finished with exit code: -11
Running Unity with the following command:
/home/max/Documents/Unity/guidewire-simulation-longitudinal-verlet.x86_64 -parameters /home/max/Temp/Praktikum/longitudinal-verlet/displacemen_0/displacemen_0_0.6157894736842106/parameters